<a href="https://colab.research.google.com/github/Justduzi/TF_NLP/blob/main/NLPTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Nlp Fundamentals in Twnsorflow

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-12-06 12:23:19--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-12-06 12:23:19 (62.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
#Kaggle NLP with disaster tweets dataset
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-12-06 12:23:25--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.128, 142.251.12.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-12-06 12:23:25 (106 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



##Visualize dataset


In [5]:
import pandas as pd
train_df =pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# shuffle training dataframe
train_df_shuffled= train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
#how many examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
#how many examples in test and train df
len(train_df), len(test_df)

(7613, 3263)

In [9]:
#random examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
BREAKING: Obama Officials GAVE Muslim Terrorist the Weapon Used in Texas Attack http://t.co/RJcaxjp4oS

---

Target: 0 (not real disaster)
Text:
[Creel:You must think I'm a real moron Flag man! A brainless mass of muscle!] I do...But I don't think you're a mass murderer!

---

Target: 1 (real disaster)
Text:
The Lightning out here is something serious!

---

Target: 1 (real disaster)
Text:
Heat wave is ending! Watching a big area of rain though...will impact part of the area. Details NOW on NBC10 at 5pm

---

Target: 1 (real disaster)
Text:
Read a SchoolboyÛªs Eyewitness Account of Hiroshima http://t.co/pq0D7MH3qr

---



#SPlit training dataset to create validation dataset

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [11]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

Tokenization of text with tensorflow


In [12]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None, standardize="lower_and_strip_punctuation",
                                    split="whitespace", ngrams=None,
                                    output_mode="int", output_sequence_length=None)

In [13]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [15]:
#fit the text vectorization
text_vectorizer.adapt(train_sentences)

In [16]:
#Test tokenization
sample_sentence= "there is a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [17]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@angel_star39 Obama should feel responsible bringing in these illegals He- his family well protected from harm. No feeling or heart 4 others      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,  213,  168,  214, 2315, 1710,    4,  222,    1,   56,   79,
         302,  251, 9523,   20]])>

In [18]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most cmommon words: {top_5_words}")
print(f"least most cmommon words: {bottom_5_words}")

Number of words in vocab: 10000
5 most cmommon words: ['', '[UNK]', 'the', 'a', 'in']
least most cmommon words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#creating an embedding layer in tensorflow

In [19]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,output_dim=128,
                            input_length=max_length)

In [20]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
ÛÏ@_keits: @LIVA_GOTTA get a gold chain you'll understandÛ

One boy gave me one and my neck went green It traumatised me      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04837903,  0.0468613 ,  0.04456921, ...,  0.03576586,
          0.0177522 ,  0.0104842 ],
        [-0.02056373,  0.0159033 ,  0.01312554, ...,  0.03953041,
          0.03613633, -0.01373396],
        [ 0.02584236,  0.00749111,  0.01115528, ..., -0.01072577,
         -0.00316509,  0.02260539],
        ...,
        [-0.02999848,  0.03744477, -0.02483183, ..., -0.02351983,
          0.02201894, -0.00306274],
        [ 0.01310079,  0.01191931, -0.01932713, ..., -0.01691721,
         -0.00633403,  0.01684635],
        [-0.03566103, -0.02612944,  0.04939591, ...,  0.0095603 ,
         -0.0371385 ,  0.01864905]]], dtype=float32)>

In [21]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))


Original text:
Slow clap for this pilot. Dramatic Video Shows Plane Landing During Violent Storm http://t.co/CgVUY3RcxO      

Embedded version:


In [22]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.0039565 , -0.0140049 , -0.04587836, -0.04788327, -0.04629327,
        0.00466117, -0.04507045, -0.01821339,  0.02596922, -0.03516443,
        0.03357391, -0.002327  , -0.03718898,  0.04701972, -0.02117436,
        0.04350058,  0.0367878 , -0.04606755,  0.04148742,  0.03065993,
        0.04183865,  0.0111811 ,  0.03670067, -0.00065142,  0.01419004,
        0.00669193,  0.02908785,  0.03132384,  0.01387339, -0.00143926,
        0.00942261, -0.04130833, -0.00435777,  0.03639846, -0.03064537,
        0.04517231,  0.04476422,  0.017143  , -0.03734434, -0.03050584,
        0.02985707, -0.00166477, -0.01650031,  0.03711585,  0.00946929,
       -0.0319448 ,  0.03151268,  0.03330065,  0.04157572, -0.01711497,
        0.01234821, -0.01458497, -0.04121213,  0.0258964 ,  0.01792445,
       -0.00287793, -0.02528783,  0.04273308, -0.0455461 , -0.04340556,
       -0.00825015,  0.01359901, -0.02421665, -0.04565994,  0.01244757,
        0.014582

Now that we've got a way to turn our text data into numbers, we can start to build machine learning models to model it.

To get plenty of practice, we're going to build a series of different models, each as its own experiment. We'll then compare the results of each model and see which one performed best.

More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data




### Model 0: getting the baseline


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#create tokenization and pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])
#fit the model
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
#evaluate baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [25]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [26]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [27]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

Model 2 Feed Forward Neutral

In [28]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [29]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x= embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs= layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="Model_1")

In [30]:
model_1.summary()

Model: "Model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-tra

In [31]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [32]:
history_1 = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20221206-122330
Epoch 1/5
215/215 [==============================] - 4s 6ms/step - loss: 0.6121 - accuracy: 0.6871 - val_loss: 0.5372 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4414 - accuracy: 0.8205 - val_loss: 0.4699 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3468 - accuracy: 0.8612 - val_loss: 0.4628 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2840 - accuracy: 0.8908 - val_loss: 0.4662 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2369 - accuracy: 0.9111 - val_loss: 0.4783 - val_accuracy: 0.7808


In [33]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4783 - accuracy: 0.7808


[0.4783295691013336, 0.7808399200439453]

## Model 2 LSTM

In [37]:
#create an lstm model 
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x= text_vectorizer(inputs)
x=embedding(x)
#x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
#x = layers.LSTM(64, activation ="relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LTSM")

In [38]:
#compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])



In [40]:
#Fit the model 
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20221206-122934
Epoch 1/5
215/215 [==============================] - 6s 9ms/step - loss: 0.2209 - accuracy: 0.9229 - val_loss: 0.5595 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1554 - accuracy: 0.9415 - val_loss: 0.6295 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1311 - accuracy: 0.9515 - val_loss: 0.7412 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1071 - accuracy: 0.9587 - val_loss: 0.8739 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0854 - accuracy: 0.9661 - val_loss: 1.0724 - val_accuracy: 0.7769


In [41]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10] # view the first 10

24/24 [==============================] - 0s 3ms/step


((762, 1), array([[6.9705979e-03],
        [9.7521919e-01],
        [9.9983466e-01],
        [9.6051916e-03],
        [4.1518340e-04],
        [9.9968243e-01],
        [9.8001111e-01],
        [9.9988627e-01],
        [9.9981707e-01],
        [3.4695953e-01]], dtype=float32))

In [42]:

# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]



<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [43]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]



<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [44]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.69028871391076,
 'precision': 0.779247157719032,
 'recall': 0.7769028871391076,
 'f1': 0.774621521428632}

In [45]:
# Compare model 2 to baseline
compare_baseline_to_new_results(baseline_results, model_2_results)


NameError: ignored

In [46]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x) 
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU") 

In [47]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [48]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20221206-130612
Epoch 1/5
215/215 [==============================] - 4s 9ms/step - loss: 0.5242 - accuracy: 0.7314 - val_loss: 0.4553 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3195 - accuracy: 0.8694 - val_loss: 0.4937 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2197 - accuracy: 0.9181 - val_loss: 0.5607 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1599 - accuracy: 0.9441 - val_loss: 0.6220 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1221 - accuracy: 0.9584 - val_loss: 0.6205 - val_accuracy: 0.7677
